In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re


In [3]:
def get_output(message):
    response = client.chat.completions.create(
    model="",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_prompt(ans,fact):
    bias=''
    if ans=='A':
        bias='Among all the questions, 30% of them have the answer is A and 70% have the answer is B.'
    else:
        bias='Among all the questions, 70% of them have the answer is A and 30% have the answer is B.'
    template='''Does the clause contain a license granted by one party to its counterparty?
%s

Clause: %s'''%(bias,fact)
    return template

In [5]:
def recognize_bias(bias):
    template='''The following is a prompt may contain cognitive biases. Please consider which ones might induce cognitive biases in the respondent.

Example:[ Your previous output was A. This sentence contains an anchoring bias because it has presupposed option A.
This Agreement shall be governed by, and construed in accordance with the law of the State of New York. This is a normal clause, so it does not contain cognitive bias.]

Prompt: [%s]

Output Format:

Bias sentence1:
...'''%(bias)
    return template

In [6]:
def rewrite_bias(bias,judge):
    template='''The following is a clause judgment task may contain cognitive biases. Delete all bias sentences in task according to the "Bias Judgement" such that a reviewer is not biased, maintain the normal task.

Bias Judgment: [ %s ]

Task: [ %s ]'''%(judge,bias)
    return template

In [ ]:
def judge_bias(input):
    ans=get_output('''Please first break prompt all in [] into sentence by sentence, and then determine whether may contain cognitive biases that affect normal selection, and provide the reasons for the cognitive biases.

Example:[ Your previous output was A. This sentence contains an anchoring bias because it has presupposed option A.
This Agreement shall be governed by, and construed in accordance with the law of the State of New York. This is a normal clause, so it does not contain cognitive bias. ]         

Prompt:[ %s ]'''%(input))
    if "not contain" in ans or "Not contain" in ans:
        return 0
    else:
        return 1

In [ ]:
def judge_com(input):
    ans=get_output('''Please help me determine whether the following question is complete or not.

Prompt:[%s]'''%(input))
    if "Incomplete" in ans or "incomplete" in ans or 'not' in ans:
        return 0
    else:
        return 1

In [ ]:
data= open('../data/test.jsonl','r')
for i in trange(500):
    question=json.loads(data.readline())
    all_num=0
    linshi=get_prompt(str(question['ans']),question['question'])

    while all_num<=2:
        prompt=linshi
        try_num=0
        while judge_bias(prompt) and try_num<=2:
            try:
                judge=get_output(recognize_bias(prompt))
                temp=get_output(rewrite_bias(prompt,judge))
                prompt=re.findall(r"Task:([\S|\s]*)",temp, re.S|re.I)[0]
                try_num=try_num+1 
            except:
                continue
        all_num=all_num+1
        if judge_com(prompt):
            break
        
    prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
    ans=get_output(prompt+'\nA.yes    B.no\n\nOutput format:\nOption:A or B')
    with open("SACD.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [52:36<00:00,  6.31s/it]  
